In [19]:
import tensorflow as tf
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg.eigen.arpack import eigsh
import scipy.sparse as sp
import pandas as pd
import pickle
from datetime import timedelta
import matplotlib.pyplot as plt
from scipy.stats.stats import pearsonr
import datetime

# Model Architecture

In [16]:
def normalize_adj(adj):
    """Symmetrically normalize adjacency matrix."""
    #adj[np.isnan(adj)] = 0.
    adj = tf.abs(adj)
    rowsum = tf.reduce_sum(adj, 1)# sum by row

    d_inv_sqrt = tf.pow(rowsum, -0.5)
   
    #d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    
    d_mat_inv_sqrt = tf.diag(d_inv_sqrt)

    return tf.matmul(tf.matmul(d_mat_inv_sqrt, adj), d_mat_inv_sqrt)

In [43]:
# Create model
def gcn(x, weights, biases, batch_size, n_input, frequency,flag, n_output_vec, num = None):
    # Hidden layer with RELU activation
    
    output_list = tf.Variable(tf.zeros([n_output_vec,1]),dtype=tf.float32) #'Tensor' object does not support item assignment, cant build Ypre
    
    if flag == 1:
        rownum = batch_size
    elif flag == 2:
        rownum = num
    elif flag == 3:
        rownum = num
    
    for i in range(rownum):
        Xtem = x[i,:]
        #Xtem = tf.reshape(x[i,:], [n_input, frequency])
        #Xtem = tf.transpose(Xtem)
        #Atem = tf.convert_to_tensor(A_whole_final, dtype=np.float32)
        #Atem1 = tf.diag(tf.ones([n_input])) # Atem not palying any roles
        #Ytem = tf.reshape(Y[i,:], [n_input, 1])
        #Atem = tf.diag(tf.ones([n_input]))
        Atem1 = 0.5*(weights['A1'] + tf.transpose(weights['A1']))#+ Atem 
        Atem1 = normalize_adj(Atem1)
        #th = tf.constant(0.01, dtype=tf.float32)
        #where = tf.subtract(Atem1, th)
        #Atem1 = tf.nn.relu(where)
        
        Z1 = tf.matmul(Atem1, Xtem) #+ tf.matmul( tf.matmul(weights['A1'], weights['A1']), Xtem)
        #layer_1 = tf.matmul(Z1, weights['h1']) 
        layer_1 = tf.add(tf.matmul(Z1, weights['h1']), biases['b1'])
        layer_1 = tf.nn.relu(layer_1)
        
        Atem2 = 0.5*(weights['A2'] + tf.transpose(weights['A2']))#+ Atem 
        Atem2 = normalize_adj(Atem2)
        
        Z2 = tf.matmul(Atem2, layer_1)
        layer_2 = tf.add(tf.matmul(Z2, weights['h2']), biases['b2'])
        layer_2 = tf.nn.relu(layer_2)
        
        Atem3 = 0.5*(weights['A3'] + tf.transpose(weights['A3']))#+ Atem 
        Atem3 = normalize_adj(Atem3)
        Z3 = tf.matmul(Atem3, layer_2)
        layer_3 = tf.add(tf.matmul(Z3, weights['h3']), biases['b3'])
        layer_3 = tf.nn.relu(layer_3)
        
        #Atem4 = 0.5*(weights['A4'] + tf.transpose(weights['A4']))#+ Atem 
        #Atem4 = normalize_adj(Atem4)
        #Z4 = tf.matmul(Atem4, layer_3)
        #layer_4 = tf.add(tf.matmul(Z4, weights['h4']), biases['b4'])
        #layer_4 = tf.nn.relu(layer_4)
        
        # flattern
        #layer_3 = tf.reshape(layer_3, [1, 272*n_hidden_vec3])
        
        #F1 = tf.add(tf.matmul(layer_3, weights['f1']), biases['bf1'])
        #F1 = tf.nn.relu(F1)
        
        #F2 = tf.add(tf.matmul(F1, weights['f2']), biases['bf2'])
        #F2 = tf.nn.relu(F2)
        
        #F3 = tf.add(tf.matmul(F1, weights['f3']), biases['bf3'])
        #out_layer = tf.reshape(F3, [272, 1])

        # Output layer with linear activation
        Z4 = layer_3#tf.matmul(Atem1, layer_1)#tf.matmul(Atem, layer_3)
        out_layer = tf.add(tf.matmul(Z4, weights['out']), biases['bout'])
        #out_layer = tf.nn.relu(out_layer)
        # weather layer 1
        #x_wea_tem = tf.reshape(x_wea[i,:], [1, 9*frequency2]) # 1 by 126
        #layer_1_wea = tf.add(tf.matmul(x_wea_tem, weights['h1_wea']), biases['b1_wea'])
        #layer_1_wea = tf.nn.relu(layer_1_wea)
        
        #out_layer_wea = tf.add(tf.matmul(layer_1_wea, weights['out_wea']), biases['bout_wea'])
        #out_layer = tf.add(out_layer, tf.reshape(out_layer_wea, [272, 1]))
        
        #print (out_layer.get_shape())
        if i ==0:
            #print (out_layer.shape)
            tem = tf.reshape(out_layer, [1, n_output_vec])
            output_list = tem
        else:
            tem = tf.reshape(out_layer, [1, n_output_vec])
            output_list = tf.concat([output_list, tem], 0)
        
        #print (tf.reduce_mean(tf.pow(output_list-out_layer, 2)))
    
    #print (output_list.get_shape())
    #print ('here!!!!!!!!!!!!!!!!')
    #output_list = tf.transpose(output_list)
    
    #print (output_list.shape)
    
    #print (output_list.get_shape())
    
    return output_list

In [50]:
def gcn_corr_final(feature_in, horizon, learning_rate, decay, batch_size, n_hidden_vec1, n_hidden_vec2, n_hidden_vec3, keep, early_stop_th, training_epochs):
   
    early_stop_k = 0 # early stop patience
    display_step = 1 # frequency of printing results
    best_val = 10000
    traing_error = 0
    test_error = 0

    tf.reset_default_graph()

    batch_size = int(batch_size)
    n_hidden_vec1 = int(n_hidden_vec1)
    n_hidden_vec2 = int(n_hidden_vec2)
    n_hidden_vec3 = int(n_hidden_vec3)
    early_stop_th = int(early_stop_th)
    training_epochs = int(training_epochs)

    # tf Graph input
    X = tf.placeholder(tf.float32, [None, sn, feature_in]) # X is the input signal
    Y = tf.placeholder(tf.float32, [None, n_output_vec]) # y is the regression output
    
    keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

    # Store layers weight & bias
    weights = {
        'h1': tf.Variable(tf.random_normal([feature_in, n_hidden_vec1])),
        'h2': tf.Variable(tf.random_normal([n_hidden_vec1, n_hidden_vec2])),
        'h3': tf.Variable(tf.random_normal([n_hidden_vec2, n_hidden_vec3])),
        'out': tf.Variable(tf.random_normal([n_hidden_vec3, horizon])),
        'A1': tf.Variable(tf.random_normal([sn, sn])),
        'A2': tf.Variable(tf.random_normal([sn, sn])),
        'A3': tf.Variable(tf.random_normal([sn, sn])),
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([sn, n_hidden_vec1])),
        'b2': tf.Variable(tf.random_normal([sn, n_hidden_vec2])), 
        'b3': tf.Variable(tf.random_normal([sn, n_hidden_vec3])),
        'bout': tf.Variable(tf.random_normal([sn, horizon])), 
    }

    # Construct model
    pred= gcn(X, weights, biases, batch_size,n_input, feature_in, 1, n_output_vec)
    
    pred = scaler.inverse_transform(pred)
    Y_true_tr = scaler.inverse_transform(Y)
    cost = tf.reduce_mean(tf.pow(pred - Y_true_tr, 2)) 

    pred_val= gcn(X, weights, biases, batch_size,n_input,feature_in, 2, n_output_vec, num_val)
    pred_val = scaler.inverse_transform(pred_val)
    Y_true_val = scaler.inverse_transform(Y)
    cost_val =  tf.reduce_mean(tf.pow(pred_val - Y_true_val, 2)) 

    pred_tes= gcn(X, weights, biases, batch_size,n_input,feature_in, 3, n_output_vec, num_test)
    pred_tes = scaler.inverse_transform(pred_tes)
    Y_true_tes = scaler.inverse_transform(Y)
    cost_tes = tf.reduce_mean(tf.pow(pred_tes - Y_true_tes, 2)) 
                                         
    optimizer = tf.train.RMSPropOptimizer(learning_rate, decay).minimize(cost)
    #optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

    # Initializing the variables
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

    with tf.Session() as sess:
        sess.run(init)

        for epoch in range(training_epochs):

            avg_cost = 0.
            total_batch = int(num_train/batch_size)

            for i in range(total_batch):
                
                _, c, preds, trueval = sess.run([optimizer, cost, pred, Y_true_tr], feed_dict={X: X_training[i*batch_size:(i+1)*batch_size,], 
                                                      Y: Y_training[i*batch_size:(i+1)*batch_size,], 
                                                              keep_prob: keep})

                avg_cost += c / total_batch 
            
             #Display logs per epoch step
            if epoch % display_step == 0:
                print ("Epoch:", '%04d' % (epoch+1), "Training RMSE=", \
                    "{:.9f}".format(np.sqrt(avg_cost)))
            # validation
            c_val = sess.run([cost_val], feed_dict={X: X_val, Y: Y_val,  keep_prob:1})
            print("Validation RMSE: ", np.sqrt(c_val[0]))
            # testing
            c_tes, pred_tes1, A1, Y_true = sess.run([cost_tes, pred_tes, weights['A1'], Y_true_tes], feed_dict={X: X_test,Y: Y_test, keep_prob: 1})
            print("Test RMSE: ", np.sqrt(c_tes))

            if c_val[0] < best_val:
                best_val = np.sqrt(c_val[0])
                # save model
                #saver.save(sess, './bikesharing_gcnn_ddgf')
                test_error = np.sqrt(c_tes)
                traing_error = np.sqrt(avg_cost)#np.sqrt(avg_cost)
                early_stop_k = 0 # reset to 0

            # update early stopping patience
            if c_val[0] >= best_val:
                early_stop_k += 1

            # threshold
            if early_stop_k == early_stop_th:
                break
            

        print("epoch is ", epoch)
        print("training error is ", traing_error)
        print("Optimization Finished! the lowest validation RMSE is ", best_val)
        print("The test RMSE is ", test_error)
    
    test_Y = Y_test
    test_error = np.sqrt(test_error)
    return best_val, predic_res,Y_true,test_error, A1

# Import Data

In [51]:
file_Name = "data/NYCBikeHourly272.pickle"
fileObject = open(file_Name,'rb') 
hourly_bike = pickle.load(fileObject) 
hourly_bike = pd.DataFrame(hourly_bike)

# Data Split into Training, Validation and Testing

In [52]:
class StandardScaler:
    """
    Standard the input
    """

    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def transform(self, data):
        return (data - self.mean) / self.std

    def inverse_transform(self, data):
        return (data * self.std) + self.mean


In [53]:
sn = 272 # station number 
feature_in = 24 # number of features at each node, e.g., bike sharing demand from past 24 hours
horizon = 1 # the length to predict, e.g., predict the future one hour bike sharing demand

X_whole = []
Y_whole = []

x_offsets = np.sort(
    np.concatenate((np.arange(-feature_in+1, 1, 1),))
)

y_offsets = np.sort(np.arange(1, 1+ horizon, 1))

min_t = abs(min(x_offsets))
max_t = abs(hourly_bike.shape[0] - abs(max(y_offsets)))  # Exclusive
for t in range(min_t, max_t):
    x_t = hourly_bike.iloc[t + x_offsets, 0:sn].values.flatten('F')
    y_t = hourly_bike.iloc[t + y_offsets, 0:sn].values.flatten('F')
    X_whole.append(x_t)
    Y_whole.append(y_t)

X_whole = np.stack(X_whole, axis=0)
Y_whole = np.stack(Y_whole, axis=0)

n_input_vec = X_whole.shape[1] # e.g., 272 * 24
n_output_vec = Y_whole.shape[1] # each row represent a result


X_whole = np.reshape(X_whole, [X_whole.shape[0], sn, feature_in])
num_samples = X_whole.shape[0]
num_test = 2000
num_val = 2000
num_train = num_samples - num_test - num_val

X_training = X_whole[:num_train, :]
Y_training = Y_whole[:num_train, :]

# shuffle the training dataset
perm = np.arange(X_training.shape[0])
np.random.shuffle(perm)
X_training = X_training[perm]
Y_training = Y_training[perm]

X_val = X_whole[num_train:num_train+num_val, :]
Y_val = Y_whole[num_train:num_train+num_val, :]

X_test = X_whole[-num_test:, :]
Y_test = Y_whole[-num_test:, :]

scaler = StandardScaler(mean=X_training.mean(), std=X_training.std())

X_training = scaler.transform(X_training)
Y_training = scaler.transform(Y_training)

X_val = scaler.transform(X_val)
Y_val = scaler.transform(Y_val)

X_test = scaler.transform(X_test)
Y_test = scaler.transform(Y_test)

# Hyperparameters

In [54]:
learning_rate = 0.005 # learning rate
decay = 0.9

n_hidden_vec1 = 100 # feature length of the first hidden layer after the graph convolution
n_hidden_vec2 = 40 # feature length of the second ...
n_hidden_vec3 = 40 # feature length of the third ...

keep = 1 # drop out probability
batchsize = 100 # batch size 
early_stop_th = 50 # early stopping threshold, if validation RMSE not dropping in continuous 50 steps, break
training_epochs = 500 # total training epochs


# Training

In [55]:
a = datetime.datetime.now()

val_error, predic_res, test_Y,test_error, A1 = gcn_corr_final(feature_in, horizon, learning_rate, decay, batchsize, n_hidden_vec1,
                                                            n_hidden_vec2, n_hidden_vec3, keep, early_stop_th, training_epochs)


b = datetime.datetime.now()

print('Total training time: ', b-a)




KeyboardInterrupt: 